# SciPy Newton-CGのテスト（多変数関数版）

多変数関数に対するNewton法のテスト。テスト用の関数はアホみたいに簡単なものをSymPyで表現し、JacobianとHessianをlambdifyしたものをscipyのminimize(method='trust-ncg')で求めています。Trust NCG はNewton共役勾配法をさらに改良した信頼領域法です。

たぶん、ほかの最適化手法でも動作します。

In [1]:
import numpy as np
import sympy as sp
from sympy.utilities import lambdify
from sympy.utilities.lambdify import lambdastr
from scipy.optimize import minimize
sp.init_printing()
from nbsupport import md

SymPyとscipy.optimize.minimizeを組み合わせる方法については、[StackOverflowへの回答](http://stackoverflow.com/questions/29443318)を見つけました。とくに多変数関数の場合の引数を渡し方が参考になりました。

-----

一見、単純そうですが大きな落とし穴があります。[StackOverflow (matrices are not aligned Error)](http://stackoverflow.com/questions/8752169)に助けられました。

小さなことなんですが、minimize が期待している Jacobian は行列ではなくベクトルです。つまり shape は $(m, 1)$ ではなく、$(m,)$でなくてはいけないようです。ドキュメントには書かれていないので本当に困りました。2時間くらい無駄にしました。

この変換は np.ndarray.flatten を使えばよいようです。

In [2]:
from sympy.abc import X, Y, Z

ARGS = [X, Y, Z]

def Formula(ARGS):
    X, Y, Z = ARGS
    return (X - 2) **2 + (Y - 4) ** 2 + (Z - 6) ** 2 + 10

def derivatives(F, ARGS):
    G = sp.Matrix([F]).jacobian(ARGS).T
    H = sp.hessian(F, ARGS)
    
    fs = [lambdify(ARGS, formula) for formula in [F, G, H]]
    
    return dict(
        function=lambda args: fs[0](*args),
        gradient=lambda args: fs[1](*args).flatten(), # Caution: gradient(...).shape must be (m,)
        hessian=lambda args: fs[2](*args))

fgh = derivatives(Formula(ARGS), ARGS) # Function/Gradient/Hessian

res = minimize(fgh['function'], np.array([[0, 0, 0]]),
               jac=fgh['gradient'], hess=fgh['hessian'],
               method='trust-ncg')

print(res)

     fun: 10.0
    hess: array([[2, 0, 0],
       [0, 2, 0],
       [0, 0, 2]])
     jac: array([ 0.,  0.,  0.])
 message: 'Optimization terminated successfully.'
    nfev: 5
    nhev: 4
     nit: 4
    njev: 5
  status: 0
 success: True
       x: array([ 2.,  4.,  6.])


In [3]:
from sympy.abc import A, B, C, X, Y, Z

CONSTS = [A, B, C]
VARS = [X, Y, Z]

def Formula(ARGS):
    (A, B, C), (X, Y, Z) = ARGS
    return (X - A) **2 + (Y - B) ** 2 + (Z - C) ** 2 + 10

def derivatives(F, VARS):
    G = sp.Matrix([F]).jacobian(VARS).T
    H = sp.hessian(F, VARS)
    
    fs = [lambdify((CONSTS, VARS), formula) for formula in [F, G, H]]
    
    
    for formula in [F, G, H]:
        print(lambdastr((CONSTS, VARS), formula))

    return dict(
        function=lambda consts: lambda vars: fs[0](consts, vars),
        gradient=lambda consts: lambda vars: fs[1](consts, vars).flatten(), # Caution: gradient(...).shape must be (m,)
        hessian=lambda  consts: lambda vars: fs[2](consts, vars))

fgh = derivatives(Formula((CONSTS, ARGS)), VARS) # Function/Gradient/Hessian

consts = [2, 4, 6]

res = minimize(fgh['function'](consts), np.array([[0, 0, 0]]),
               jac=fgh['gradient'](consts),
               hess=fgh['hessian'](consts),
               method='trust-ncg')

print(res)

lambda _0,_1: (lambda A,B,C,X,Y,Z: ((-A + X)**2 + (-B + Y)**2 + (-C + Z)**2 + 10))(*list(__flatten_args__([_0,_1])))
lambda _0,_1: (lambda A,B,C,X,Y,Z: (MutableDenseMatrix([[-2*A + 2*X], [-2*B + 2*Y], [-2*C + 2*Z]])))(*list(__flatten_args__([_0,_1])))
lambda _0,_1: (lambda A,B,C,X,Y,Z: (MutableDenseMatrix([[2, 0, 0], [0, 2, 0], [0, 0, 2]])))(*list(__flatten_args__([_0,_1])))
     fun: 10.0
    hess: array([[2, 0, 0],
       [0, 2, 0],
       [0, 0, 2]])
     jac: array([ 0.,  0.,  0.])
 message: 'Optimization terminated successfully.'
    nfev: 5
    nhev: 4
     nit: 4
    njev: 5
  status: 0
 success: True
       x: array([ 2.,  4.,  6.])


In [4]:
# 某、参考にしたサンプル実装

def objectiveFunction(x):
    # 目的関数
    # f(x,y) = 3*x1^2 + 2*x1*x2 + x2^2
    f = 3*x[0]**2 + 2*x[0]*x[1] + x[1]**2
    return f

def gradient(x):
    # 勾配ベクトル
    g = np.array([6*x[0] + 2*x[1], 2*x[0] + 2*x[1]])
    return g

def Hessian(x):
    # ヘッセ行列
    h = np.array([[6, 2], [2, 2]])
    return h

x0 = [10.0, 10.0]
res = minimize(objectiveFunction, x0, jac=gradient, hess=Hessian, method="trust-ncg")
print(res)

     fun: 1.4791141972893971e-31
    hess: array([[6, 2],
       [2, 2]])
     jac: array([ -4.44089210e-16,   4.44089210e-16])
 message: 'Optimization terminated successfully.'
    nfev: 6
    nhev: 5
     nit: 5
    njev: 6
  status: 0
 success: True
       x: array([ -2.22044605e-16,   4.44089210e-16])


In [5]:
def Objective(x, y):
    f = 3*x**2 + 2*x*y + y**2
    return f

def Gradient(x, y):
    g = np.array([6*x + 2*y, 2*x + 2*y])
    return g

def Hessian(x, y):
    h = np.array([[6, 2], [2, 2]])
    return h

x0 = [10.0, 10.0]
res = minimize(lambda args: Objective(*args), x0,
               jac=lambda args: Gradient(*args),
               hess=lambda args: Hessian(*args), method="trust-ncg")
print(res)

     fun: 1.4791141972893971e-31
    hess: array([[6, 2],
       [2, 2]])
     jac: array([ -4.44089210e-16,   4.44089210e-16])
 message: 'Optimization terminated successfully.'
    nfev: 6
    nhev: 5
     nit: 5
    njev: 6
  status: 0
 success: True
       x: array([ -2.22044605e-16,   4.44089210e-16])
